# Interactive Geospatial Analysis Exploration

This notebook provides an interactive environment to explore the results of the rail corridor flood vulnerability analysis. 
Using `geopandas` and `folium`, we can visualize the spatial distribution of vulnerability and infrastructure gaps.

In [ ]:
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
import mapclassify

# Set working directory to project root if running from notebooks folder
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
    
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Load Analysis Results
gpkg_path = 'data/outputs_final/analysis_segments.gpkg'

if os.path.exists(gpkg_path):
    gdf = gpd.read_file(gpkg_path)
    print(f"Loaded {len(gdf)} segments.")
    print(f"CRS: {gdf.crs}")
else:
    print(f"Error: Could not find {gpkg_path}. Please run the analysis script first.")

In [ ]:
# Basic Statistics
if 'gdf' in locals():
    display(gdf.head())
    display(gdf[['vuln_mean', 'density_sqft_per_acre', 'gap_index']].describe())

## Interactive Vulnerability Map

The map below shows the calculated Flood Vulnerability Score (0-10) for each rail segment. 
- **Red**: High Vulnerability
- **Blue**: Low Vulnerability

In [ ]:
if 'gdf' in locals():
    # Create interactive map
    m_vuln = gdf.explore(
        column='vuln_mean',
        cmap='RdYlBu_r',
        tiles='CartoDB positron',
        tooltip=['segment_id', 'vuln_mean', 'vuln_class', 'imperv_mean', 'slope_mean'],
        popup=True,
        legend_kwds={'caption': 'Flood Vulnerability Score'}
    )
    display(m_vuln)

## Infrastructure Gap Analysis

This map highlights "Priority Areas" where the Gap Index is high.
**Gap Index** = Vulnerability Score - Infrastructure Adequacy Score

Positive values indicate areas where vulnerability exceeds the protection provided by current infrastructure.

In [ ]:
if 'gdf' in locals():
    # Filter for positive gaps
    gap_gdf = gdf[gdf['gap_index'] > 0]
    
    m_gap = gap_gdf.explore(
        column='gap_index',
        cmap='Reds',
        tiles='CartoDB positron',
        tooltip=['segment_id', 'gap_index', 'vuln_mean', 'density_sqft_per_acre'],
        legend_kwds={'caption': 'Infrastructure Gap Index'}
    )
    display(m_gap)

## Correlation Analysis

Visualizing the relationship between vulnerability and infrastructure density.

In [ ]:
if 'gdf' in locals():
    plt.figure(figsize=(10, 6))
    plt.scatter(gdf['vuln_mean'], gdf['density_sqft_per_acre'], alpha=0.5)
    plt.title('Vulnerability vs. Infrastructure Density')
    plt.xlabel('Vulnerability Score')
    plt.ylabel('Density (sq ft/acre)')
    plt.grid(True, alpha=0.3)
    plt.show()